In [1]:
import grequests as asynhttp
import pandas as pd
import json

### Data Collection and Cleaning

In [2]:
capitales = {
    "Africa": ["El Cairo", "Abuya", "Nairobi", "Addis Abeba", "Pretoria"],
    "America": ["Washington D. C.", "Ottawa", "Ciudad de México", "Brasilia", "Buenos Aires",],
    "Asia": ["Pekín", "Tokio", "Nueva Delhi", "Seúl", "Riad"],
    "Europa": ["Madrid", "París", "Berlín", "Roma", "Londres"],
    "Oceania": ["Canberra", "Wellington", "Port Moresby", "Suva", "Apia"],
}
key="030c86ad50fe0b9ad9817deeb4fc2825"

In [3]:
localizaciones={
    "name": [],
    "lat": [],
    "lon": []
}

rs=[]
for continente in capitales:
    for ciudad in capitales[continente]:
        url=f"http://api.openweathermap.org/geo/1.0/direct?q={ciudad}&appid={key}"
        rs.append(asynhttp.get(url))
        
futures = asynhttp.map(rs)
for futuro in futures:
    response=futuro.json()
    localizaciones["name"].append(response[0]["name"])
    localizaciones["lat"].append(response[0]["lat"])
    localizaciones["lon"].append(response[0]["lon"])

In [4]:
df_localizaciones=pd.DataFrame(localizaciones).set_index("name")
df_localizaciones

,lat,lon
name,,
Cairo,30.044388,31.235726
Abuja,9.064331,7.489297
Nairobi,-1.303264,36.826384
Addis Ababa,9.010793,38.761252
Pretoria,-25.745928,28.187910
Washington,38.895037,-77.036543
Ottawa,45.420878,-75.690111
Mexico City,19.432630,-99.133178
Brasília,-15.793404,-47.882317


In [5]:
rs=[]
for entrada in df_localizaciones.values:
    url=f"https://api.openweathermap.org/data/2.5/weather?lat={entrada[0]}&lon={entrada[1]}&appid={key}&units=metric&lang=sp"
    rs.append(asynhttp.get(url))

tiempo=[]
futures = asynhttp.map(rs)
for futuro in futures:
    tiempo.append(json.loads(futuro.text))

In [6]:
for ciudad in tiempo:
    del ciudad["coord"]
    for i, e in enumerate(ciudad["weather"]):
        del ciudad["weather"][i]["id"]
        del ciudad["weather"][i]["icon"]
    del ciudad["sys"]
    del ciudad["timezone"]
    del ciudad["id"]
    del ciudad["name"]
    del ciudad["cod"]

In [7]:
df_tiempo = pd.json_normalize(tiempo, "weather", ["base", ["main", "feels_like"],
    ["main", "temp_min"],["main", "temp_max"],["main", "pressure"],["main", "humidity"],
    ["main", "sea_level"],["main", "grnd_level"], "visibility", ["wind", "speed"], ["wind", "deg"], 
    ["clouds", "all"], "dt"], sep="-")

In [8]:
df_tiempo = df_localizaciones.reset_index().join(df_tiempo)

In [13]:
df_tiempo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype                        
---  ------           --------------  -----                        
 0   name             25 non-null     object                       
 1   lat              25 non-null     float64                      
 2   lon              25 non-null     float64                      
 3   main             25 non-null     object                       
 4   description      25 non-null     object                       
 5   base             25 non-null     object                       
 6   main-feels_like  25 non-null     float64                      
 7   main-temp_min    25 non-null     float64                      
 8   main-temp_max    25 non-null     float64                      
 9   main-pressure    25 non-null     int64                        
 10  main-humidity    25 non-null     int64                        
 11  main-sea

In [10]:
df_tiempo = df_tiempo.astype({"main-feels_like": "float",
                              "main-temp_min": "float", 
                              "main-temp_max": "float", 
                              "main-pressure": "int",
                              "main-humidity": "int",
                              "main-sea_level": "int", 
                              "main-grnd_level": "int", 
                              "visibility": "int", 
                              "wind-speed": "float", 
                              "wind-deg": "int", 
                              "clouds-all": "int"})
df_tiempo["dt"] = pd.to_datetime(df_tiempo["dt"], unit="s", utc=True).dt.tz_convert("Europe/Madrid")

In [11]:
df_tiempo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype                        
---  ------           --------------  -----                        
 0   name             25 non-null     object                       
 1   lat              25 non-null     float64                      
 2   lon              25 non-null     float64                      
 3   main             25 non-null     object                       
 4   description      25 non-null     object                       
 5   base             25 non-null     object                       
 6   main-feels_like  25 non-null     float64                      
 7   main-temp_min    25 non-null     float64                      
 8   main-temp_max    25 non-null     float64                      
 9   main-pressure    25 non-null     int64                        
 10  main-humidity    25 non-null     int64                        
 11  main-sea

In [12]:
df_tiempo.sample(5)

,name,lat,lon,main,description,base,main-feels_like,main-temp_min,main-temp_max,main-pressure,main-humidity,main-sea_level,main-grnd_level,visibility,wind-speed,wind-deg,clouds-all,dt
11,Tokyo,35.676860,139.763895,Clear,cielo claro,stations,-7.65,-2.06,-2.06,1032,15,1032,1027,10000,5.26,310,2,2025-12-02 18:42:21+01:00
8,Brasília,-15.793404,-47.882317,Clear,cielo claro,stations,19.09,19.94,19.94,1015,42,1015,764,10000,3.09,80,0,2025-12-02 18:41:15+01:00
15,Madrid,40.416705,-3.703582,Clouds,nubes,stations,20.81,21.96,21.96,1017,23,1017,945,10000,2.83,132,98,2025-12-02 18:39:21+01:00
19,London,51.507322,-0.127647,Clouds,muy nuboso,stations,9.38,8.74,10.08,1014,92,1014,1006,10000,1.54,140,75,2025-12-02 18:37:30+01:00
3,Addis Ababa,9.010793,38.761252,Clouds,nubes dispersas,stations,11.72,13.10,13.10,1018,48,1018,775,10000,2.79,67,25,2025-12-02 18:39:21+01:00


In [14]:
df_tiempo.to_csv("tiempo_global.csv")

### Data Analysis
- Which cities are the hottest and coldest right now?
- Is there a relationship between temperature and humidity?
- Which continent seems to have the windiest cities?
- What’s the most common weather description (e.g., clear, cloudy, rainy)?